In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta
from faker import Faker

# Initialize Faker
fake = Faker()

# Define options
user_categories = ['Surgeon', 'Nurse', 'Medical Student', 'Technician']
locations = ['USA', 'UK', 'India', 'Germany', 'Canada', 'Australia']
experience_levels = ['Novice', 'Intermediate', 'Expert']
brands = ['MedTools', 'HealthPlus', 'MediLine', 'SurgiPro', 'CareSurg']
specializations = ['Orthopedic', 'Cardiac', 'Neurosurgery', 'General', 'Pediatric']
budget_ranges = ['Low', 'Medium', 'High']
tools = [
    'Scalpel-A', 'Clamp-B', 'Retractor-X', 'Scissors-C', 'Syringe-F',
    'Forceps-M', 'NeedleHolder-Z', 'Drill-Y', 'Suction-E', 'Stapler-N'
]

# Generate the data
def generate_user_data(num_users=2000):
    data = []
    for i in range(num_users):
        user_id = f"U{i+1:04d}"
        category = random.choice(user_categories)
        location = random.choice(locations)
        experience = random.choice(experience_levels)
        specialization = random.choice(specializations)
        purchase_count = random.randint(3, 6)
        purchases = '|'.join(random.sample(tools, purchase_count))
        last_purchase = fake.date_between(start_date='-1y', end_date='today')
        brand = random.choice(brands)
        budget = random.choice(budget_ranges)

        data.append([
            user_id, category, location, experience, purchases,
            last_purchase, brand, specialization, budget
        ])

    columns = [
        'userID', 'userCategory', 'location', 'experienceLevel',
        'previousPurchases', 'lastPurchaseDate', 'preferredBrand',
        'specialization', 'budgetRange'
    ]

    return pd.DataFrame(data, columns=columns)

# Run and save
df = generate_user_data()
df.to_excel("surgical_tool_recommendation_users.xlsx", index=False)
print("Done! Saved to 'surgical_tool_recommendation_users.xlsx'")


Done! Saved to 'surgical_tool_recommendation_users.xlsx'


In [ ]:
import pandas as pd

# Tool-price mapping
tool_data = {
    'Scalpel-A': 45,
    'Clamp-B': 35,
    'Retractor-X': 120,
    'Scissors-C': 40,
    'Syringe-F': 15,
    'Forceps-M': 60,
    'NeedleHolder-Z': 70,
    'Drill-Y': 200,
    'Suction-E': 150,
    'Stapler-N': 90
}

# Convert to DataFrame
tool_df = pd.DataFrame(list(tool_data.items()), columns=['toolName', 'priceUSD'])

# Save as CSV
tool_df.to_excel("surgical_tool_prices.xlsx", index=False)
print("✅ File 'surgical_tool_prices.xlsx' created successfully.")


✅ File 'surgical_tool_prices.xlsx' created successfully.


# 5. Budget Based recommendation system

In [ ]:
import pandas as pd

# Load user and tool data
users_df = pd.read_excel("surgical_tool_recommendation_users.xlsx")
tools_df = pd.read_excel("surgical_tool_prices.xlsx")

# Budget filter logic
budget_price_map = {
    'Low': lambda p: p <= 50,
    'Medium': lambda p: 51 <= p <= 100,
    'High': lambda p: p > 100
}

# Tool-price lookup
tool_price_dict = tools_df.set_index("toolName")["priceUSD"].to_dict()

# Function to recommend tools
def recommend_tools_for_user(user_row, top_k=3):
    budget = user_row['budgetRange']
    purchased = user_row['previousPurchases'].split('|')

    tools_in_budget = [
        tool for tool, price in tool_price_dict.items()
        if budget_price_map[budget](price)
    ]

    recommendations = [tool for tool in tools_in_budget if tool not in purchased]
    return recommendations[:top_k]

# --- Main block ---
def get_user_recommendation(user_id):
    user_row = users_df[users_df['userID'] == user_id]

    if user_row.empty:
        print(f"❌ No user found with ID '{user_id}'")
        return

    user_row = user_row.iloc[0]

    print("🧾 User Profile:")
    print(user_row.to_string())
    print("\n🔍 Recommended Tools (Budget-Sensitive):")

    recs = recommend_tools_for_user(user_row)
    for i, tool in enumerate(recs, 1):
        print(f"{i}. {tool} (${tool_price_dict[tool]})")

# 🔧 Replace with a real userID from your dataset
get_user_recommendation("U1115")

for i in range(10):
  get_user_recommendation(users_df['userID'][i])
  print("________________________________________________________________\n",)


🧾 User Profile:
userID                                                           U1115
userCategory                                           Medical Student
location                                                            UK
experienceLevel                                           Intermediate
previousPurchases    Drill-Y|Suction-E|Stapler-N|NeedleHolder-Z|Syr...
lastPurchaseDate                                   2024-10-09 00:00:00
preferredBrand                                              HealthPlus
specialization                                                 General
budgetRange                                                        Low

🔍 Recommended Tools (Budget-Sensitive):
1. Scalpel-A ($45)
2. Clamp-B ($35)
3. Scissors-C ($40)
🧾 User Profile:
userID                                         U0001
userCategory                                 Surgeon
location                                       India
experienceLevel                         Intermediate
previousPurchases   

# Utilizing more columns to make the recommendations more targeted

In [ ]:
import pandas as pd

# Load user and tool data
users_df = pd.read_excel("surgical_tool_recommendation_users.xlsx")
tools_df = pd.read_excel("surgical_tool_prices.xlsx")

# Price range mapping
budget_price_map = {
    'Low': lambda p: p <= 50,
    'Medium': lambda p: 51 <= p <= 100,
    'High': lambda p: p > 100
}

# Tool-price lookup
tool_price_dict = tools_df.set_index("toolName")["priceUSD"].to_dict()

# Tool-category mapping
tool_category_map = {
    'Scalpel-A': ['Surgeon'],
    'Clamp-B': ['Surgeon', 'Nurse', 'Dentist'],
    'Retractor-X': ['Surgeon'],
    'Scissors-C': ['Surgeon', 'Nurse'],
    'Syringe-F': ['Nurse', 'Dentist'],
    'Forceps-M': ['Surgeon'],
    'NeedleHolder-Z': ['Surgeon'],
    'Drill-Y': ['Dentist'],
    'Suction-E': ['Dentist'],
    'Stapler-N': ['Surgeon']
}

# Recommendation function (with more relaxed rules)
def recommend_tools_for_user(user_row, top_k=3):
    budget = user_row['budgetRange']
    purchased = user_row['previousPurchases'].split('|')
    category = user_row['userCategory']

    # Hold all potential recommendations
    recommendations = []

    # First, try to recommend tools that match both budget and category
    for tool, price in tool_price_dict.items():
        if tool in purchased:
            continue  # Skip already purchased tools

        # Match by budget and category
        if budget_price_map[budget](price) and category in tool_category_map.get(tool, []):
            recommendations.append(tool)

    # If not enough recommendations, relax filtering
    if len(recommendations) < top_k:
        for tool, price in tool_price_dict.items():
            if tool in purchased:
                continue  # Skip already purchased tools

            # Prioritize tools within budget, but allow more flexibility for expensive ones
            if budget_price_map[budget](price) or len(recommendations) >= top_k:
                recommendations.append(tool)

            if len(recommendations) >= top_k:
                break  # Stop once we have enough recommendations

    return recommendations[:top_k]

# Main function to show profile + recommendations
def get_user_recommendation(user_id):
    user_row = users_df[users_df['userID'] == user_id]

    if user_row.empty:
        print(f"❌ No user found with ID '{user_id}'")
        return

    user_row = user_row.iloc[0]

    print("🧾 User Profile:")
    print(user_row.to_string())
    print("\n🔍 Recommended Tools (Filtered by Budget + Role):")

    recs = recommend_tools_for_user(user_row)
    if recs:
        for i, tool in enumerate(recs, 1):
            print(f"{i}. {tool} (${tool_price_dict[tool]})")
    else:
        print("No suitable recommendations found.")

# 🔧 Replace with a real userID from your dataset
for i in range(10):
  get_user_recommendation(users_df['userID'][i])
  print("________________________________________________________________\n",)


🧾 User Profile:
userID                                         U0001
userCategory                                 Surgeon
location                                       India
experienceLevel                         Intermediate
previousPurchases    Forceps-M|Retractor-X|Stapler-N
lastPurchaseDate                 2025-02-10 00:00:00
preferredBrand                            HealthPlus
specialization                          Neurosurgery
budgetRange                                     High

🔍 Recommended Tools (Filtered by Budget + Role):
1. Drill-Y ($200)
2. Suction-E ($150)
________________________________________________________________

🧾 User Profile:
userID                                                           U0002
userCategory                                                     Nurse
location                                                           USA
experienceLevel                                                 Novice
previousPurchases    Stapler-N|Clamp-B|Drill-Y|Forcep